# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax
If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.
If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [2]:
import csv
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [3]:
# Set the URL of the paginated webpage that you want to scrape
url = "https://www.airlinequality.com/airline-reviews/british-airways"

# Initialize an empty list to store the data that you scrape
data = []

# Setting the initial page number and the increment that you want to use to paginate through the webpage
page_num = 1
page_incr = 1
page_size = 100
# maximum number of pages to be scraped
max_pages = 20

# Set the URL of the webpage to be scraped 
paginated_url = f"{url}/page/{page_num}/?sortby=post_date%3ADesc&pagesize={page_size}"

# A while loop to paginate through the webpage and scrape the data
while page_num <= max_pages:

    print(f"Scraping page {page_num}")

    # A GET request to the paginated URL
    response = requests.get(paginated_url)

    # Parsing the response using BeautifulSoup
    parsed_content = BeautifulSoup(response.text, "html.parser")

    # Finding all the elements on the page that contain the data to be scraped
    elements = parsed_content.find_all("div",class_ = "body")

    # Looping through the elements and extract the data that you want to scrape
    for element in elements:
        header = element.find("h2",class_ = "text_header").text.replace("\n", " ")
        sub_header = element.find("h3",class_ = "text_sub_header").text.replace("\n", " ")
        content = element.find("div",class_ = "text_content").text.replace("\n", " ")
        
        data.append([header,sub_header,content])

    # Increasing the page number and setting the paginated URL to the new page
    page_num += page_incr
    paginated_url = f"{url}/page/{page_num}/?sortby=post_date%3ADesc&pagesize={page_size}"

    print(f"   ---> {len(data)} total reviews")

Scraping page 1
   ---> 100 total reviews
Scraping page 2
   ---> 200 total reviews
Scraping page 3
   ---> 300 total reviews
Scraping page 4
   ---> 400 total reviews
Scraping page 5
   ---> 500 total reviews
Scraping page 6
   ---> 600 total reviews
Scraping page 7
   ---> 700 total reviews
Scraping page 8
   ---> 800 total reviews
Scraping page 9
   ---> 900 total reviews
Scraping page 10
   ---> 1000 total reviews
Scraping page 11
   ---> 1100 total reviews
Scraping page 12
   ---> 1200 total reviews
Scraping page 13
   ---> 1300 total reviews
Scraping page 14
   ---> 1400 total reviews
Scraping page 15
   ---> 1500 total reviews
Scraping page 16
   ---> 1600 total reviews
Scraping page 17
   ---> 1700 total reviews
Scraping page 18
   ---> 1800 total reviews
Scraping page 19
   ---> 1900 total reviews
Scraping page 20
   ---> 2000 total reviews


The loops above collected 2000 reviews by iterating through the paginated pages on the website.

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

In [4]:
#Coverting the list data into a dataframe
df = pd.DataFrame(data)
df.columns = ["REVIEW","PERSONAL INFO","CONTENT"]

#Removing unwanted text(first text preprocessing)
df.replace(re.compile(r'\s*✅ Trip Verified \|\s*'), '', inplace=True)
df


,REVIEW,PERSONAL INFO,CONTENT
0,"""It was a nightmare""",Guadalupe Carlos-Alarcon (United States) 18t...,Not Verified | They changed our Flights from ...
1,"""Abysmal service""",Patrick Sparks (United States) 18th April 2023,Not Verified | At Copenhagen the most chaotic...
2,"""trained to give you the runaround""",T Cayle (United States) 17th April 2023,Worst experience of my life trying to deal wit...
3,"""they only had one choice of meal""",1 reviews Andrew Pybus (Hong Kong) 17th A...,Due to code sharing with Cathay Pacific I was ...
4,"""relentless BA cost cutting""",M Edwards (United Kingdom) 16th April 2023,LHR check in was quick at the First Wing and q...
...,...,...,...
1995,"""great cabin crew on both sectors""",C Kay (United Kingdom) 1st August 2016,✅ Verified Review | London Heathrow to Nassau...
1996,"""provided reasonable cabin service""",E Smyth (United Kingdom) 1st August 2016,✅ Verified Review | Miami to London Heathrow ...
1997,"""not direct but it was well worth it.""",Ben Thain (Qatar) 31st July 2016,✅ Verified Review | Doha to Paris CDG via Bah...
1998,"""main course was simply dreadful""",38 reviews Richard Callis (Brazil) 30th J...,✅ Verified Review | Sao Paulo to Heathrow with...


In [6]:
#Saving data into a csv
df.to_csv(r".\Datasets\BA_reviews.csv")

In [15]:
sentiment_analysis_df = df.drop(["REVIEW","PERSONAL INFO"], axis=1)
sentiment_analysis_df.replace(re.compile(r'\s*✅ Verified Review \|\s*'), '', inplace=True)
sentiment_analysis_df.replace(re.compile(r'\s*Not Verified \|\s*'), '', inplace=True)
sentiment_analysis_df

,CONTENT
0,They changed our Flights from Brussels to Lond...
1,At Copenhagen the most chaotic ticket counter ...
2,Worst experience of my life trying to deal wit...
3,Due to code sharing with Cathay Pacific I was ...
4,LHR check in was quick at the First Wing and q...
...,...
1995,London Heathrow to Nassau return with British ...
1996,Miami to London Heathrow with British Airways....
1997,"Doha to Paris CDG via Bahrain and London, prem..."
1998,Sao Paulo to Heathrow with British Airways. TA...


In [16]:
# Save the DataFrame to a CSV file
sentiment_analysis_df.to_csv("sentiment_content.csv", index=False)